# Xception

In [1]:
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, SeparableConv2D, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D, Dense, Add, Dropout
from keras.models import Model
from keras.optimizers import Adam

## The Model

In [2]:
def ConvBN(X_input, filters, kernel_size, strides, padding = 'valid'):
    X = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(X_input)
    return BatchNormalization()(X)

In [3]:
def SeparableConvBN(X_input, filters, kernel_size):
    X = SeparableConv2D(filters=filters, kernel_size=kernel_size, padding='same')(X_input)
    return BatchNormalization()(X)

### 1. Entry Flow

In [4]:
def EntryFlow(X_input) :
    X = ConvBN(X_input, 32, (3,3), (2,2))
    X = Activation('relu')(X)
    X = ConvBN(X, 64, (3,3), (1,1))
    X = Activation('relu')(X)
    
    X1 = SeparableConvBN(X, 128, (3,3))
    X1 = Activation('relu')(X1)
    X1 = SeparableConvBN(X1, 128, (3,3))
    X1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding = 'same')(X1)
    X2 = ConvBN(X, 128, (1,1), (2,2))
    X = Add()([X1, X2])
    
    X1 = Activation('relu')(X)
    X1 = SeparableConvBN(X1, 256, (3,3))
    X1 = Activation('relu')(X1)
    X1 = SeparableConvBN(X1, 256, (3,3))
    X1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding = 'same')(X1)
    X2 = ConvBN(X, 256, (1,1), (2,2))
    X = Add()([X1, X2])
    
    X1 = Activation('relu')(X)
    X1 = SeparableConvBN(X1, 728, (3,3))
    X1 = Activation('relu')(X1)
    X1 = SeparableConvBN(X1, 728, (3,3))
    X1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding = 'same')(X1)
    X2 = ConvBN(X, 728, (1,1), (2,2))
    X = Add()([X1, X2])
    
    return X

### 2. Middle Flow

In [5]:
def MiddleFlow(X_input):
    X = Activation('relu')(X_input)
    X = SeparableConvBN(X, 728, (3,3))
    X = Activation('relu')(X)
    X = SeparableConvBN(X, 728, (3,3))
    X = Activation('relu')(X)
    X = SeparableConvBN(X, 728, (3,3))
    X = Add()([X, X_input])
    return X

### 3. Exit Flow

In [6]:
def ExitFlow(X_input):
    X1 = Activation('relu')(X_input)
    X1 = SeparableConvBN(X1, 728, (3,3))
    X1 = Activation('relu')(X1)
    X1 = SeparableConvBN(X1, 1024, (3,3))
    X1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding = 'same')(X1)
    X2 = ConvBN(X_input, 1024, (1,1), (2,2))
    X = Add()([X1, X2])
    
    X = SeparableConvBN(X, 1536, (3,3))
    X = Activation('relu')(X)
    X = SeparableConvBN(X, 2048, (3,3))
    X = Activation('relu')(X)
    
    X = GlobalAveragePooling2D()(X)
    return X

### Complete Model Architecture

In [7]:
X_input = Input(shape=(299,299,3))

X = EntryFlow(X_input)

for i in range(8):
    X = MiddleFlow(X)

X = ExitFlow(X)

X = Dropout(rate=0.5)(X)
X = Dense(units = 1000, activation='softmax')(X)

xception = Model(inputs=X_input, outputs=X)
xception.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [8]:
xception.save('Xception.keras')

## Data Preparation

In [14]:
TRAIN_DIR = "../input/imagenetmini-1000/imagenet-mini/train/"
VAL_DIR = "../input/imagenetmini-1000/imagenet-mini/val/"
batch_size = 32

from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=True)

train_gen = train_generator.flow_from_directory(TRAIN_DIR, 
                                               target_size=(299, 299), 
                                               batch_size=batch_size, 
                                               class_mode='categorical')
val_gen = val_generator.flow_from_directory(VAL_DIR, 
                                           target_size=(229, 229),
                                           batch_size=batch_size, 
                                           class_mode='categorical')

Found 34745 images belonging to 1000 classes.
Found 3923 images belonging to 1000 classes.


## Model Training

In [15]:
opt = Adam(learning_rate=0.045)
xception.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
from livelossplot import PlotLossesKeras

xception.fit_generator(train_gen, 
                      epochs=5, 
                      steps_per_epoch=34745//batch_size, 
                      validation_data=val_gen, 
                      validation_steps=3923//batch_size,
                      callbacks=[PlotLossesKeras()], 
                      verbose=1)

Epoch 1/5
